In [ ]:
import numpy as np
import string
import pickle
from collections import Counter

# =============================
# ENHANCED HMM TRAINING
# =============================

ALPHABET = list(string.ascii_uppercase)
SYM2IDX = {c: i for i, c in enumerate(ALPHABET)}
IDX2SYM = {i: c for c, i in SYM2IDX.items()}

def normalize_rows(M, eps=1e-12):
    """Normalize matrix rows to sum to 1"""
    M = M + eps
    return M / M.sum(axis=1, keepdims=True)

def normalize_vec(v, eps=1e-12):
    """Normalize vector to sum to 1"""
    v = v + eps
    return v / v.sum()


# ========== DATA LOADING ==========
def load_corpus(path):
    """Load and preprocess corpus with statistics"""
    words = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            w = line.strip().upper()
            w = ''.join([c for c in w if c in SYM2IDX])
            if len(w) > 0:
                words.append(w)

    # Compute statistics
    lengths = [len(w) for w in words]
    print(f"✅ Loaded {len(words)} words from {path}")
    print(f"   Length range: {min(lengths)}-{max(lengths)}, avg: {np.mean(lengths):.1f}")

    return words

def words_to_observation_sequences(words):
    """Convert words to observation sequences"""
    return [[SYM2IDX[c] for c in w] for w in words]


# ========== SMART INITIALIZATION ==========
def init_params_smart(words, n_states=26, n_obs=26):
    """
    Initialize HMM parameters using corpus statistics
    Much better than random initialization!
    """
    # Count letter frequencies for initial state prior
    first_letters = Counter([w[0] for w in words])
    pi = np.array([first_letters.get(ALPHABET[i], 0) for i in range(n_states)], dtype=float)
    pi = normalize_vec(pi)

    # Count bigram transitions for state transition matrix
    A = np.ones((n_states, n_states))  # Smoothing
    for word in words:
        for i in range(len(word) - 1):
            curr_idx = SYM2IDX[word[i]]
            next_idx = SYM2IDX[word[i + 1]]
            A[curr_idx, next_idx] += 1
    A = normalize_rows(A)

    # Initialize emission matrix with identity + smoothing
    # States tend to emit their corresponding letter
    B = np.ones((n_states, n_obs)) * 0.1
    for i in range(min(n_states, n_obs)):
        B[i, i] = 10.0  # Strong diagonal
    B = normalize_rows(B)

    print("✅ Smart initialization using corpus statistics")
    print(f"   Top initial letters: {[(ALPHABET[i], first_letters.get(ALPHABET[i], 0)) for i in range(5)]}")

    return pi, A, B


# ========== FORWARD-BACKWARD WITH STABILITY ==========
def forward_backward_single(obs, pi, A, B):
    """
    Scaled forward-backward algorithm with numerical stability
    """
    T = len(obs)
    N = len(pi)

    alpha = np.zeros((T, N))
    beta = np.zeros((T, N))
    c = np.zeros(T)

    # Forward pass with scaling
    alpha[0] = pi * B[:, obs[0]]
    c[0] = np.sum(alpha[0])
    if c[0] > 0:
        alpha[0] /= c[0]
    else:
        c[0] = 1.0

    for t in range(1, T):
        alpha[t] = (alpha[t-1] @ A) * B[:, obs[t]]
        c[t] = np.sum(alpha[t])
        if c[t] > 0:
            alpha[t] /= c[t]
        else:
            c[t] = 1.0

    # Backward pass with scaling
    beta[-1] = 1.0
    for t in range(T - 2, -1, -1):
        beta[t] = A @ (B[:, obs[t + 1]] * beta[t + 1])
        if c[t] > 0:
            beta[t] /= c[t]

    # Compute gamma (state posteriors)
    gamma = alpha * beta
    gamma_sum = np.sum(gamma, axis=1, keepdims=True)
    gamma = np.where(gamma_sum > 0, gamma / gamma_sum, 1.0 / N)

    # Compute xi (transition posteriors)
    xi = np.zeros((T - 1, N, N))
    for t in range(T - 1):
        x = (alpha[t][:, None] * A) * (B[:, obs[t + 1]] * beta[t + 1])[None, :]
        xi_sum = np.sum(x)
        if xi_sum > 0:
            xi[t] = x / xi_sum
        else:
            xi[t] = A / N  # Fallback

    # Log-likelihood
    log_likelihood = np.sum(np.log(c + 1e-300))

    return gamma, xi, log_likelihood


# ========== EM ALGORITHM ==========
def estep(obs_seqs, pi, A, B, verbose=False):
    """E-step: compute expected sufficient statistics"""
    N, M = len(pi), B.shape[1]
    pi_exp = np.zeros(N)
    A_exp = np.zeros((N, N))
    B_exp = np.zeros((N, M))
    total_ll = 0

    for idx, obs in enumerate(obs_seqs):
        gamma, xi, ll = forward_backward_single(obs, pi, A, B)
        total_ll += ll

        pi_exp += gamma[0]
        A_exp += np.sum(xi, axis=0)

        for t, o in enumerate(obs):
            B_exp[:, o] += gamma[t]

        if verbose and (idx + 1) % 1000 == 0:
            print(f"   Processed {idx + 1}/{len(obs_seqs)} sequences...")

    return pi_exp, A_exp, B_exp, total_ll


def mstep(pi_exp, A_exp, B_exp, smoothing=1e-6):
    """M-step: update parameters with smoothing"""
    pi_new = normalize_vec(pi_exp + smoothing)
    A_new = normalize_rows(A_exp + smoothing)
    B_new = normalize_rows(B_exp + smoothing)
    return pi_new, A_new, B_new


def train_hmm_baum_welch(obs_seqs, words, n_states=26, n_obs=26,
                         max_iters=20, convergence_tol=1.0, verbose=True):
    """
    Train HMM using Baum-Welch with early stopping

    Args:
        obs_seqs: Observation sequences
        words: Original words (for smart init)
        n_states: Number of hidden states
        n_obs: Number of observation symbols
        max_iters: Maximum iterations
        convergence_tol: Stop if LL improvement < this value
        verbose: Print progress
    """
    # Smart initialization
    pi, A, B = init_params_smart(words, n_states, n_obs)

    prev_ll = None
    best_ll = -np.inf
    best_params = None
    no_improvement = 0

    print(f"\n{'='*50}")
    print(f"🚀 TRAINING HMM: {len(obs_seqs)} sequences, {max_iters} max iterations")
    print(f"{'='*50}")

    for i in range(max_iters):
        print(f"\n📊 Iteration {i+1}/{max_iters}")

        # E-step
        pi_exp, A_exp, B_exp, total_ll = estep(obs_seqs, pi, A, B, verbose=verbose)

        # M-step
        pi, A, B = mstep(pi_exp, A_exp, B_exp)

        # Compute per-word average
        avg_ll = total_ll / len(obs_seqs)

        print(f"   Total LL: {total_ll:.2f}")
        print(f"   Avg LL per word: {avg_ll:.4f}")

        if prev_ll is not None:
            improvement = total_ll - prev_ll
            print(f"   Improvement: {improvement:.2f}")

            # Check for convergence
            if improvement < convergence_tol:
                no_improvement += 1
                print(f"   ⚠️  Small improvement ({no_improvement}/3)")
                if no_improvement >= 3:
                    print(f"\n✅ CONVERGED after {i+1} iterations!")
                    break
            else:
                no_improvement = 0

        # Track best model
        if total_ll > best_ll:
            best_ll = total_ll
            best_params = (pi.copy(), A.copy(), B.copy())
            print(f"   ⭐ New best model!")

        prev_ll = total_ll

    print(f"\n{'='*50}")
    print(f"✅ TRAINING COMPLETE")
    print(f"   Final LL: {prev_ll:.2f}")
    print(f"   Best LL: {best_ll:.2f}")
    print(f"{'='*50}\n")

    # Return best parameters
    return best_params if best_params else (pi, A, B)


# ========== MAIN TRAINING PIPELINE ==========
def main():
    # Load corpus
    words = load_corpus("corpus.txt")
    obs_seqs = words_to_observation_sequences(words)

    print(f"\n📈 Total sequences: {len(obs_seqs)}")

    # Use more data for better learning (5000+ words recommended)
    # Remove slice or increase for full corpus
    train_size = min(5000, len(obs_seqs))
    print(f"   Training on: {train_size} words")

    train_words = words[:train_size]
    train_obs = obs_seqs[:train_size]

    # Train model
    pi, A, B = train_hmm_baum_welch(
        train_obs,
        train_words,
        n_states=26,
        n_obs=26,
        max_iters=20,
        convergence_tol=1.0,
        verbose=True
    )

    # Save model
    model_data = {
        "pi": pi,
        "A": A,
        "B": B,
        "train_size": train_size,
        "alphabet": ALPHABET,
        "sym2idx": SYM2IDX,
        "idx2sym": IDX2SYM
    }

    with open("hmm_trained_v1.pkl", "wb") as f:
        pickle.dump(model_data, f)

    print("📦 Model saved as hmm_trained_v1.pkl")

    # Print model insights
    print("\n🔍 MODEL INSIGHTS:")
    print(f"   Most likely initial states: {[ALPHABET[i] for i in np.argsort(pi)[-5:][::-1]]}")
    print(f"   Emission matrix sparsity: {np.sum(B < 0.01) / B.size * 100:.1f}%")

    return pi, A, B


if __name__ == "__main__":
    main()

✅ Loaded 50000 words from corpus.txt
   Length range: 1-24, avg: 9.5

📈 Total sequences: 50000
   Training on: 5000 words
✅ Smart initialization using corpus statistics
   Top initial letters: [('A', 385), ('B', 223), ('C', 453), ('D', 252), ('E', 167)]

🚀 TRAINING HMM: 5000 sequences, 20 max iterations

📊 Iteration 1/20
   Processed 1000/5000 sequences...
   Processed 2000/5000 sequences...
   Processed 3000/5000 sequences...
   Processed 4000/5000 sequences...
   Processed 5000/5000 sequences...
   Total LL: -123633.86
   Avg LL per word: -24.7268
   ⭐ New best model!

📊 Iteration 2/20
   Processed 1000/5000 sequences...
   Processed 2000/5000 sequences...
   Processed 3000/5000 sequences...
   Processed 4000/5000 sequences...
   Processed 5000/5000 sequences...
   Total LL: -119885.19
   Avg LL per word: -23.9770
   Improvement: 3748.66
   ⭐ New best model!

📊 Iteration 3/20
   Processed 1000/5000 sequences...
   Processed 2000/5000 sequences...
   Processed 3000/5000 sequences...
 

In [ ]:
import numpy as np
import string
import pickle
from collections import Counter, defaultdict

# =============================
# HMM ACCURACY TESTING
# =============================

ALPHABET = list(string.ascii_uppercase)
SYM2IDX = {c: i for i, c in enumerate(ALPHABET)}
IDX2SYM = {i: c for c, i in SYM2IDX.items()}


# ========== LOAD MODEL ==========
def load_hmm_model(path="hmm_trained_v1.pkl"):
    """Load trained HMM model"""
    with open(path, "rb") as f:
        model = pickle.load(f)
    print(f"✅ Loaded HMM model from {path}")
    print(f"   Training size: {model.get('train_size', 'unknown')}")
    return model['pi'], model['A'], model['B']


# ========== LOAD TEST DATA ==========
def load_test_corpus(path):
    """Load test corpus"""
    words = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            w = line.strip().upper()
            w = ''.join([c for c in w if c in SYM2IDX])
            if len(w) > 0:
                words.append(w)

    print(f"✅ Loaded {len(words)} test words from {path}")
    return words


# ========== VITERBI ALGORITHM ==========
def viterbi(obs, pi, A, B):
    """
    Viterbi algorithm to find most likely hidden state sequence
    Returns: best state sequence and log probability
    """
    T = len(obs)
    N = len(pi)

    # Initialize
    delta = np.zeros((T, N))
    psi = np.zeros((T, N), dtype=int)

    # t=0
    delta[0] = np.log(pi + 1e-300) + np.log(B[:, obs[0]] + 1e-300)

    # Forward pass
    for t in range(1, T):
        for j in range(N):
            # Find max probability transition to state j
            prob = delta[t-1] + np.log(A[:, j] + 1e-300)
            psi[t, j] = np.argmax(prob)
            delta[t, j] = prob[psi[t, j]] + np.log(B[j, obs[t]] + 1e-300)

    # Backtrack
    states = np.zeros(T, dtype=int)
    states[-1] = np.argmax(delta[-1])
    for t in range(T-2, -1, -1):
        states[t] = psi[t+1, states[t+1]]

    log_prob = np.max(delta[-1])

    return states, log_prob


# ========== FORWARD ALGORITHM ==========
def forward_log_likelihood(obs, pi, A, B):
    """
    Compute log-likelihood of observation sequence using forward algorithm
    """
    T = len(obs)
    N = len(pi)

    alpha = np.zeros((T, N))
    c = np.zeros(T)

    # Initialize
    alpha[0] = pi * B[:, obs[0]]
    c[0] = np.sum(alpha[0])
    if c[0] > 0:
        alpha[0] /= c[0]
    else:
        c[0] = 1.0

    # Forward pass with scaling
    for t in range(1, T):
        alpha[t] = (alpha[t-1] @ A) * B[:, obs[t]]
        c[t] = np.sum(alpha[t])
        if c[t] > 0:
            alpha[t] /= c[t]
        else:
            c[t] = 1.0

    # Log-likelihood
    log_likelihood = np.sum(np.log(c + 1e-300))

    return log_likelihood


# ========== PREDICTION ACCURACY ==========
def predict_next_letter(prefix, pi, A, B, top_k=5):
    """
    Given a prefix, predict the most likely next letters
    Returns: list of (letter, probability) tuples
    """
    if len(prefix) == 0:
        # Use initial distribution and emission
        probs = pi @ B
    else:
        # Convert prefix to observations
        obs = [SYM2IDX[c] for c in prefix]

        # Run forward algorithm to get state distribution at end
        T = len(obs)
        N = len(pi)
        alpha = np.zeros((T, N))

        alpha[0] = pi * B[:, obs[0]]
        alpha[0] /= np.sum(alpha[0])

        for t in range(1, T):
            alpha[t] = (alpha[t-1] @ A) * B[:, obs[t]]
            alpha[t] /= np.sum(alpha[t])

        # Predict next letter
        state_dist = alpha[-1] @ A  # Next state distribution
        probs = state_dist @ B  # Next letter probabilities

    # Get top k predictions
    top_indices = np.argsort(probs)[-top_k:][::-1]
    predictions = [(ALPHABET[i], probs[i]) for i in top_indices]

    return predictions


def test_next_letter_prediction(words, pi, A, B, sample_size=1000):
    """
    Test accuracy of predicting next letter given prefix
    Tests at different prefix lengths
    """
    print("\n" + "="*60)
    print("🎯 NEXT-LETTER PREDICTION ACCURACY")
    print("="*60)

    np.random.seed(42)
    test_words = np.random.choice(words, min(sample_size, len(words)), replace=False)

    results = {}

    for prefix_len in [1, 2, 3, 4, 5]:
        correct_top1 = 0
        correct_top3 = 0
        correct_top5 = 0
        total = 0

        for word in test_words:
            if len(word) <= prefix_len:
                continue

            prefix = word[:prefix_len]
            true_next = word[prefix_len]

            predictions = predict_next_letter(prefix, pi, A, B, top_k=5)
            pred_letters = [p[0] for p in predictions]

            if pred_letters[0] == true_next:
                correct_top1 += 1
            if true_next in pred_letters[:3]:
                correct_top3 += 1
            if true_next in pred_letters[:5]:
                correct_top5 += 1

            total += 1

        if total > 0:
            acc_top1 = correct_top1 / total * 100
            acc_top3 = correct_top3 / total * 100
            acc_top5 = correct_top5 / total * 100

            results[prefix_len] = {
                'top1': acc_top1,
                'top3': acc_top3,
                'top5': acc_top5,
                'total': total
            }

            print(f"\n📏 Prefix length {prefix_len} ({total} samples):")
            print(f"   Top-1 accuracy: {acc_top1:.2f}%")
            print(f"   Top-3 accuracy: {acc_top3:.2f}%")
            print(f"   Top-5 accuracy: {acc_top5:.2f}%")

    return results


# ========== PERPLEXITY ==========
def calculate_perplexity(words, pi, A, B):
    """
    Calculate perplexity - lower is better
    Perplexity = exp(-avg log likelihood)
    """
    print("\n" + "="*60)
    print("📊 PERPLEXITY CALCULATION")
    print("="*60)

    total_ll = 0
    total_letters = 0

    for word in words:
        obs = [SYM2IDX[c] for c in word]
        ll = forward_log_likelihood(obs, pi, A, B)
        total_ll += ll
        total_letters += len(word)

    avg_ll = total_ll / total_letters
    perplexity = np.exp(-avg_ll)

    print(f"\n   Total log-likelihood: {total_ll:.2f}")
    print(f"   Avg LL per letter: {avg_ll:.4f}")
    print(f"   Perplexity: {perplexity:.2f}")
    print(f"   (Lower perplexity = better model)")

    return perplexity, avg_ll


# ========== HANGMAN SIMULATION ==========
def simulate_hangman_guess(word, pi, A, B, known_letters=set()):
    """
    Simulate guessing letters for hangman
    Returns best letter guess given current knowledge
    """
    # Create pattern (known letters visible, others hidden)
    pattern = ['_' if c not in known_letters else c for c in word]

    # Score each possible letter
    letter_scores = {}

    for letter in ALPHABET:
        if letter in known_letters:
            continue

        # Try this letter in each position
        score = 0
        for pos, p in enumerate(pattern):
            if p == '_':
                # Predict letter at this position
                prefix = ''.join([c for c in pattern[:pos] if c != '_'])
                predictions = predict_next_letter(prefix, pi, A, B, top_k=26)

                # Find this letter's rank
                for rank, (pred_letter, prob) in enumerate(predictions):
                    if pred_letter == letter:
                        score += prob
                        break

        letter_scores[letter] = score

    # Return best guess
    if letter_scores:
        best_letter = max(letter_scores, key=letter_scores.get)
        return best_letter, letter_scores[best_letter]
    return None, 0


def test_hangman_performance(words, pi, A, B, sample_size=100, max_guesses=10):
    """
    Test how well the model performs at hangman
    """
    print("\n" + "="*60)
    print("🎮 HANGMAN SIMULATION")
    print("="*60)

    np.random.seed(42)
    test_words = np.random.choice(words, min(sample_size, len(words)), replace=False)

    total_correct = 0
    total_guesses = 0
    win_count = 0

    for word in test_words:
        known = set()
        guesses = 0

        while guesses < max_guesses:
            if all(c in known for c in word):
                win_count += 1
                break

            guess, score = simulate_hangman_guess(word, pi, A, B, known)
            if guess is None:
                break

            guesses += 1
            known.add(guess)

            if guess in word:
                total_correct += 1

            total_guesses += 1

    accuracy = total_correct / total_guesses * 100 if total_guesses > 0 else 0
    win_rate = win_count / len(test_words) * 100

    print(f"\n   Games played: {len(test_words)}")
    print(f"   Win rate: {win_rate:.2f}%")
    print(f"   Letter guess accuracy: {accuracy:.2f}%")
    print(f"   Avg guesses per game: {total_guesses / len(test_words):.2f}")

    return win_rate, accuracy


# ========== MAIN TESTING ==========
def main():
    print("\n" + "="*60)
    print("🧪 HMM MODEL ACCURACY TESTING")
    print("="*60)

    # Load model
    pi, A, B = load_hmm_model("hmm_trained.pkl")

    # Load test data
    test_words = load_test_corpus("test.txt")

    print(f"\n📋 Test dataset: {len(test_words)} words")

    # Test 1: Perplexity (overall fit)
    perplexity, avg_ll = calculate_perplexity(test_words, pi, A, B)

    # Test 2: Next-letter prediction accuracy
    pred_results = test_next_letter_prediction(test_words, pi, A, B, sample_size=2000)

    # Test 3: Hangman simulation
    win_rate, guess_acc = test_hangman_performance(test_words, pi, A, B, sample_size=200)

    # Summary
    print("\n" + "="*60)
    print("📈 SUMMARY")
    print("="*60)
    print(f"\n✅ Perplexity: {perplexity:.2f} (lower is better)")
    print(f"✅ Avg log-likelihood per letter: {avg_ll:.4f}")

    if pred_results:
        avg_top1 = np.mean([r['top1'] for r in pred_results.values()])
        avg_top3 = np.mean([r['top3'] for r in pred_results.values()])
        print(f"✅ Avg Top-1 prediction accuracy: {avg_top1:.2f}%")
        print(f"✅ Avg Top-3 prediction accuracy: {avg_top3:.2f}%")

    print(f"✅ Hangman win rate: {win_rate:.2f}%")
    print(f"✅ Hangman guess accuracy: {guess_acc:.2f}%")

    print("\n" + "="*60)


if __name__ == "__main__":
    main()


🧪 HMM MODEL ACCURACY TESTING
✅ Loaded HMM model from hmm_trained.pkl
   Training size: 5000
✅ Loaded 2000 test words from test.txt

📋 Test dataset: 2000 words

📊 PERPLEXITY CALCULATION

   Total log-likelihood: -48215.19
   Avg LL per letter: -2.5009
   Perplexity: 12.19
   (Lower perplexity = better model)

🎯 NEXT-LETTER PREDICTION ACCURACY

📏 Prefix length 1 (2000 samples):
   Top-1 accuracy: 26.80%
   Top-3 accuracy: 56.55%
   Top-5 accuracy: 76.20%

📏 Prefix length 2 (1998 samples):
   Top-1 accuracy: 16.27%
   Top-3 accuracy: 38.29%
   Top-5 accuracy: 56.76%

📏 Prefix length 3 (1989 samples):
   Top-1 accuracy: 17.65%
   Top-3 accuracy: 39.77%
   Top-5 accuracy: 56.26%

📏 Prefix length 4 (1952 samples):
   Top-1 accuracy: 19.93%
   Top-3 accuracy: 45.75%
   Top-5 accuracy: 61.78%

📏 Prefix length 5 (1861 samples):
   Top-1 accuracy: 18.75%
   Top-3 accuracy: 45.19%
   Top-5 accuracy: 62.92%

🎮 HANGMAN SIMULATION

   Games played: 200
   Win rate: 3.00%
   Letter guess accuracy: 4

In [ ]:
import numpy as np
import string
import pickle
from collections import Counter, defaultdict
import random

ALPHABET = list(string.ascii_uppercase)
SYM2IDX = {c: i for i, c in enumerate(ALPHABET)}
IDX2SYM = {i: c for c, i in SYM2IDX.items()}


class HMMFeatureExtractor:
    """
    Extracts HMM-based features for RL state representation
    """

    def __init__(self, pi, A, B):
        self.pi = pi
        self.A = A
        self.B = B
        self.N = len(pi)

    def get_letter_probabilities(self, pattern, guessed_letters):
        """
        Get HMM probability distribution over all 26 letters
        Returns: vector of 26 probabilities
        """
        length = len(pattern)

        # Convert pattern to observation indices (-1 for unknown)
        obs = []
        unknown_positions = []
        for i, c in enumerate(pattern):
            if c == '_':
                obs.append(-1)
                unknown_positions.append(i)
            else:
                obs.append(SYM2IDX[c])

        if not unknown_positions:
            return np.zeros(26)

        # Run forward-backward
        alpha = self._forward(obs)
        beta = self._backward(obs)

        # Compute marginal probabilities for each letter
        letter_probs = np.zeros(26)

        for pos in unknown_positions:
            gamma = alpha[pos] * beta[pos]
            gamma = gamma / (np.sum(gamma) + 1e-300)

            # Accumulate emission probabilities
            for letter_idx in range(26):
                letter_probs[letter_idx] += np.sum(gamma * self.B[:, letter_idx])

        # Normalize
        total = np.sum(letter_probs)
        if total > 0:
            letter_probs = letter_probs / total

        # Zero out already guessed letters
        for letter in guessed_letters:
            letter_probs[SYM2IDX[letter]] = 0

        return letter_probs

    def get_state_distribution(self, pattern):
        """
        Get HMM hidden state distribution given pattern
        Returns: vector of state probabilities
        """
        obs = []
        for c in pattern:
            if c == '_':
                obs.append(-1)
            else:
                obs.append(SYM2IDX[c])

        alpha = self._forward(obs)
        return alpha[-1] / (np.sum(alpha[-1]) + 1e-300)

    def _forward(self, obs):
        """Forward algorithm with unknown observations"""
        T = len(obs)
        alpha = np.zeros((T, self.N))

        # Initialize
        if obs[0] == -1:
            alpha[0] = self.pi
        else:
            alpha[0] = self.pi * self.B[:, obs[0]]
        alpha[0] = alpha[0] / (np.sum(alpha[0]) + 1e-300)

        # Forward pass
        for t in range(1, T):
            alpha[t] = alpha[t-1] @ self.A
            if obs[t] != -1:
                alpha[t] = alpha[t] * self.B[:, obs[t]]
            alpha[t] = alpha[t] / (np.sum(alpha[t]) + 1e-300)

        return alpha

    def _backward(self, obs):
        """Backward algorithm with unknown observations"""
        T = len(obs)
        beta = np.zeros((T, self.N))
        beta[-1] = 1.0

        for t in range(T-2, -1, -1):
            if obs[t+1] != -1:
                beta[t] = self.A @ (self.B[:, obs[t+1]] * beta[t+1])
            else:
                beta[t] = self.A @ beta[t+1]
            beta[t] = beta[t] / (np.sum(beta[t]) + 1e-300)

        return beta


class HangmanQLearningAgent:
    """
    Q-Learning agent for Hangman that uses HMM features
    """

    def __init__(self, hmm_extractor, learning_rate=0.1, discount=0.95,
                 epsilon=0.1, epsilon_decay=0.995, epsilon_min=0.01):
        self.hmm = hmm_extractor
        self.alpha = learning_rate
        self.gamma = discount
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min

        # Q-table: state -> action -> value
        self.Q = defaultdict(lambda: np.zeros(26))

        # For tracking performance
        self.training_stats = {
            'games_played': 0,
            'games_won': 0,
            'total_wrong_guesses': 0,
            'total_repeated_guesses': 0
        }

    def get_state_key(self, pattern, guessed_letters, wrong_guesses_left):
        """
        Create state representation combining:
        - Pattern (compressed)
        - Guessed letters (as bitmask)
        - Wrong guesses remaining
        - HMM top predictions (discretized)
        """
        # Pattern compression: length + number of known letters + positions
        length = len(pattern)
        known_count = sum(1 for c in pattern if c != '_')

        # Get HMM probabilities
        hmm_probs = self.hmm.get_letter_probabilities(pattern, guessed_letters)

        # Get top 5 HMM predictions (discretized)
        top_5_indices = np.argsort(hmm_probs)[-5:][::-1]
        top_5_letters = tuple(ALPHABET[i] for i in top_5_indices)

        # Guessed letters as sorted tuple
        guessed_tuple = tuple(sorted(guessed_letters))

        # Create composite state key
        state_key = (
            length,
            known_count,
            pattern,  # Full pattern for exact matching
            guessed_tuple,
            wrong_guesses_left,
            top_5_letters  # HMM guidance
        )

        return state_key

    def get_state_key_simple(self, pattern, guessed_letters, wrong_guesses_left):
        """
        Simpler state representation for smaller state space
        """
        # Get HMM probabilities
        hmm_probs = self.hmm.get_letter_probabilities(pattern, guessed_letters)

        # Discretize HMM probs into bins
        top_3_indices = np.argsort(hmm_probs)[-3:][::-1]
        top_3_letters = tuple(ALPHABET[i] for i in top_3_indices)

        # Pattern features
        length = len(pattern)
        known_count = sum(1 for c in pattern if c != '_')
        known_ratio = int(known_count / length * 10)  # 0-10

        state_key = (
            length,
            known_ratio,
            wrong_guesses_left,
            top_3_letters,
            tuple(sorted(guessed_letters))
        )

        return state_key

    def choose_action(self, state_key, guessed_letters, hmm_probs, training=True):
        """
        Choose action using epsilon-greedy with HMM guidance
        """
        available_letters = [i for i in range(26) if ALPHABET[i] not in guessed_letters]

        if not available_letters:
            return None

        # Epsilon-greedy exploration
        if training and random.random() < self.epsilon:
            # Explore: weighted random by HMM probs
            probs = np.array([hmm_probs[i] for i in available_letters])
            if np.sum(probs) > 0:
                probs = probs / np.sum(probs)
                action = np.random.choice(available_letters, p=probs)
            else:
                action = random.choice(available_letters)
        else:
            # Exploit: choose best Q-value among available
            q_values = self.Q[state_key]

            # Combine Q-values with HMM probs for better decisions
            combined_scores = np.zeros(26)
            for i in available_letters:
                # 70% Q-value, 30% HMM probability
                combined_scores[i] = 0.7 * q_values[i] + 0.3 * hmm_probs[i]

            action = np.argmax(combined_scores)

            # If Q-value is 0 (unvisited), fall back to HMM
            if q_values[action] == 0:
                action = np.argmax(hmm_probs)

        return action

    def update_q_value(self, state, action, reward, next_state, done):
        """
        Q-learning update: Q(s,a) += α[r + γ max Q(s',a') - Q(s,a)]
        """
        current_q = self.Q[state][action]

        if done:
            max_next_q = 0
        else:
            max_next_q = np.max(self.Q[next_state])

        # Q-learning update
        new_q = current_q + self.alpha * (reward + self.gamma * max_next_q - current_q)
        self.Q[state][action] = new_q

    def train_episode(self, word, max_wrong=6):
        """
        Train on a single game of Hangman
        """
        pattern = ['_'] * len(word)
        guessed_letters = set()
        wrong_guesses = 0
        repeated_guesses = 0

        episode_history = []  # (state, action, reward)

        while wrong_guesses < max_wrong:
            # Check win condition
            if '_' not in pattern:
                # Won! Backpropagate rewards
                final_reward = 100 - (wrong_guesses * 5) - (repeated_guesses * 2)
                self._backpropagate_rewards(episode_history, final_reward)
                self.training_stats['games_won'] += 1
                return True, wrong_guesses, repeated_guesses

            # Get current state
            pattern_str = ''.join(pattern)
            hmm_probs = self.hmm.get_letter_probabilities(pattern_str, guessed_letters)
            state = self.get_state_key_simple(pattern_str, guessed_letters, max_wrong - wrong_guesses)

            # Choose action
            action = self.choose_action(state, guessed_letters, hmm_probs, training=True)
            if action is None:
                break

            letter = ALPHABET[action]

            # Check for repeated guess
            if letter in guessed_letters:
                repeated_guesses += 1
                reward = -10  # Heavy penalty for repeated guess
                episode_history.append((state, action, reward))
                continue

            guessed_letters.add(letter)

            # Check if letter is in word
            if letter in word:
                # Correct guess
                for i, c in enumerate(word):
                    if c == letter:
                        pattern[i] = letter

                reward = 5  # Reward for correct guess
            else:
                # Wrong guess
                wrong_guesses += 1
                reward = -5  # Penalty for wrong guess

            episode_history.append((state, action, reward))

        # Lost game
        final_reward = -(wrong_guesses * 5) - (repeated_guesses * 2) - 50
        self._backpropagate_rewards(episode_history, final_reward)

        return False, wrong_guesses, repeated_guesses

    def _backpropagate_rewards(self, episode_history, final_reward):
        """
        Backpropagate rewards through episode
        """
        # Update Q-values for all state-action pairs in episode
        for i, (state, action, immediate_reward) in enumerate(episode_history):
            # Calculate discounted future reward
            remaining_steps = len(episode_history) - i - 1
            discounted_final = final_reward * (self.gamma ** remaining_steps)
            total_reward = immediate_reward + discounted_final

            # Get next state
            if i < len(episode_history) - 1:
                next_state = episode_history[i + 1][0]
                done = False
            else:
                next_state = state
                done = True

            self.update_q_value(state, action, total_reward, next_state, done)

    def train(self, word_list, episodes=5000):
        """
        Train the Q-learning agent on a list of words
        """
        print(f"\n🎓 Training Q-Learning Agent for {episodes} episodes...")
        print(f"   Word list size: {len(word_list)}")

        for episode in range(episodes):
            # Sample random word
            word = random.choice(word_list).upper()
            word = ''.join([c for c in word if c in SYM2IDX])

            if len(word) == 0:
                continue

            # Train episode
            won, wrong, repeated = self.train_episode(word)

            # Update stats
            self.training_stats['games_played'] += 1
            self.training_stats['total_wrong_guesses'] += wrong
            self.training_stats['total_repeated_guesses'] += repeated

            # Decay epsilon
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

            # Print progress
            if (episode + 1) % 500 == 0:
                games = self.training_stats['games_played']
                wins = self.training_stats['games_won']
                win_rate = wins / games * 100 if games > 0 else 0
                avg_wrong = self.training_stats['total_wrong_guesses'] / games
                avg_repeated = self.training_stats['total_repeated_guesses'] / games

                print(f"\n   Episode {episode + 1}/{episodes}")
                print(f"   Win rate: {win_rate:.2f}%")
                print(f"   Avg wrong guesses: {avg_wrong:.2f}")
                print(f"   Avg repeated guesses: {avg_repeated:.2f}")
                print(f"   Epsilon: {self.epsilon:.4f}")
                print(f"   Q-table size: {len(self.Q)}")

        print(f"\n✅ Training complete!")
        print(f"   Final win rate: {self.training_stats['games_won'] / self.training_stats['games_played'] * 100:.2f}%")
        print(f"   Q-table states: {len(self.Q)}")

    def play_game(self, word, max_wrong=6, verbose=False):
        """
        Play a single game (testing/evaluation)
        """
        pattern = ['_'] * len(word)
        guessed_letters = set()
        wrong_guesses = 0
        repeated_guesses = 0

        if verbose:
            print(f"\n🎮 Playing: {word}")

        while wrong_guesses < max_wrong:
            if '_' not in pattern:
                if verbose:
                    print(f"   ✅ WON! Wrong: {wrong_guesses}, Repeated: {repeated_guesses}")
                return True, wrong_guesses, repeated_guesses

            pattern_str = ''.join(pattern)
            hmm_probs = self.hmm.get_letter_probabilities(pattern_str, guessed_letters)
            state = self.get_state_key_simple(pattern_str, guessed_letters, max_wrong - wrong_guesses)

            action = self.choose_action(state, guessed_letters, hmm_probs, training=False)
            if action is None:
                break

            letter = ALPHABET[action]

            if letter in guessed_letters:
                repeated_guesses += 1
                if verbose:
                    print(f"   ❌ Repeated: {letter}")
                continue

            guessed_letters.add(letter)

            if letter in word:
                for i, c in enumerate(word):
                    if c == letter:
                        pattern[i] = letter
                if verbose:
                    print(f"   ✓ {letter}: {''.join(pattern)}")
            else:
                wrong_guesses += 1
                if verbose:
                    print(f"   ✗ {letter}: {''.join(pattern)} ({wrong_guesses}/{max_wrong})")

        if verbose:
            print(f"   ❌ LOST! Wrong: {wrong_guesses}, Repeated: {repeated_guesses}")
        return False, wrong_guesses, repeated_guesses

    def evaluate(self, test_words, max_wrong=6):
        """
        Evaluate agent on test set
        """
        print(f"\n📊 Evaluating on {len(test_words)} words...")

        wins = 0
        total_wrong = 0
        total_repeated = 0

        for word in test_words:
            won, wrong, repeated = self.play_game(word, max_wrong)
            if won:
                wins += 1
            total_wrong += wrong
            total_repeated += repeated

        success_rate = wins / len(test_words)
        final_score = (success_rate * 2000) - (total_wrong * 5) - (total_repeated * 2)

        print(f"\n📈 RESULTS:")
        print(f"   Games played: {len(test_words)}")
        print(f"   Wins: {wins}")
        print(f"   Success rate: {success_rate * 100:.2f}%")
        print(f"   Total wrong guesses: {total_wrong}")
        print(f"   Total repeated guesses: {total_repeated}")
        print(f"   Avg wrong per game: {total_wrong / len(test_words):.2f}")
        print(f"   Avg repeated per game: {total_repeated / len(test_words):.2f}")
        print(f"\n   🏆 FINAL SCORE: {final_score:.2f}")

        return success_rate, total_wrong, total_repeated, final_score

    def save(self, path="q_learning_agent.pkl"):
        """Save Q-table and parameters"""
        data = {
            'Q': dict(self.Q),
            'alpha': self.alpha,
            'gamma': self.gamma,
            'epsilon': self.epsilon,
            'stats': self.training_stats
        }
        with open(path, 'wb') as f:
            pickle.dump(data, f)
        print(f"✅ Saved agent to {path}")

    def load(self, path="q_learning_agent.pkl"):
        """Load Q-table and parameters"""
        with open(path, 'rb') as f:
            data = pickle.load(f)
        self.Q = defaultdict(lambda: np.zeros(26), data['Q'])
        self.alpha = data['alpha']
        self.gamma = data['gamma']
        self.epsilon = data['epsilon']
        self.training_stats = data['stats']
        print(f"✅ Loaded agent from {path}")


# ========== MAIN USAGE ==========
def main():
    print("="*60)
    print("🤖 HMM + Q-Learning Hangman Agent")
    print("="*60)

    # Load HMM model
    print("\n1️⃣ Loading HMM model...")
    with open("hmm_trained.pkl", "rb") as f:
        model = pickle.load(f)
    pi, A, B = model['pi'], model['A'], model['B']

    # Create HMM feature extractor
    hmm_extractor = HMMFeatureExtractor(pi, A, B)

    # Load training words
    print("\n2️⃣ Loading training words...")
    with open("corpus.txt", 'r', encoding='utf-8') as f:
        train_words = [line.strip().upper() for line in f]
    train_words = [''.join([c for c in w if c in SYM2IDX]) for w in train_words]
    train_words = [w for w in train_words if len(w) > 0]
    print(f"   Loaded {len(train_words)} training words")

    # Create Q-learning agent
    agent = HangmanQLearningAgent(
        hmm_extractor,
        learning_rate=0.1,
        discount=0.95,
        epsilon=0.3,
        epsilon_decay=0.995,
        epsilon_min=0.01
    )

    # Train agent
    print("\n3️⃣ Training agent...")
    agent.train(train_words, episodes=10000)

    # Save agent
    agent.save("hangman_qlearning_agent.pkl")

    # Load test words
    print("\n4️⃣ Loading test words...")
    with open("test.txt", 'r', encoding='utf-8') as f:
        test_words = [line.strip().upper() for line in f]
    test_words = [''.join([c for c in w if c in SYM2IDX]) for w in test_words]
    test_words = [w for w in test_words if len(w) > 0]
    print(f"   Loaded {len(test_words)} test words")

    # Evaluate
    print("\n5️⃣ Evaluating agent...")
    agent.evaluate(test_words[:2000], max_wrong=6)

    print("\n" + "="*60)


if __name__ == "__main__":
    main()

🤖 HMM + Q-Learning Hangman Agent

1️⃣ Loading HMM model...

2️⃣ Loading training words...
   Loaded 50000 training words

3️⃣ Training agent...

🎓 Training Q-Learning Agent for 10000 episodes...
   Word list size: 50000

   Episode 500/10000
   Win rate: 27.20%
   Avg wrong guesses: 5.38
   Avg repeated guesses: 0.00
   Epsilon: 0.0245
   Q-table size: 4670

   Episode 1000/10000
   Win rate: 29.10%
   Avg wrong guesses: 5.35
   Avg repeated guesses: 0.00
   Epsilon: 0.0100
   Q-table size: 8789

   Episode 1500/10000
   Win rate: 30.53%
   Avg wrong guesses: 5.31
   Avg repeated guesses: 0.00
   Epsilon: 0.0100
   Q-table size: 12607

   Episode 2000/10000
   Win rate: 31.65%
   Avg wrong guesses: 5.29
   Avg repeated guesses: 0.00
   Epsilon: 0.0100
   Q-table size: 16153

   Episode 2500/10000
   Win rate: 32.64%
   Avg wrong guesses: 5.26
   Avg repeated guesses: 0.00
   Epsilon: 0.0100
   Q-table size: 19601

   Episode 3000/10000
   Win rate: 32.23%
   Avg wrong guesses: 5.26
   

In [ ]:
import numpy as np
import string
import pickle
from collections import deque
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

ALPHABET = list(string.ascii_uppercase)
SYM2IDX = {c: i for i, c in enumerate(ALPHABET)}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device: {device}")


class SimpleHMM:
    """Simple cached HMM"""

    def __init__(self, pi, A, B):
        self.pi = pi
        self.A = A
        self.B = B
        self.N = len(pi)
        self.cache = {}

    def get_probs(self, pattern, guessed):
        """Get letter probabilities"""
        key = (pattern, tuple(sorted(guessed)))
        if key in self.cache:
            return self.cache[key]

        # Parse pattern
        obs = []
        unknowns = []
        for i, c in enumerate(pattern):
            if c == '_':
                obs.append(-1)
                unknowns.append(i)
            else:
                obs.append(SYM2IDX[c])

        if not unknowns:
            result = np.zeros(26)
            self.cache[key] = result
            return result

        # Forward-backward
        alpha = self._forward(obs)
        beta = self._backward(obs)

        probs = np.zeros(26)
        for pos in unknowns:
            gamma = alpha[pos] * beta[pos]
            gamma = gamma / (gamma.sum() + 1e-10)
            probs += gamma @ self.B

        probs = probs / (probs.sum() + 1e-10)

        # Zero guessed
        for letter in guessed:
            probs[SYM2IDX[letter]] = 0

        # Renormalize
        if probs.sum() > 0:
            probs = probs / probs.sum()

        self.cache[key] = probs
        return probs

    def _forward(self, obs):
        T = len(obs)
        alpha = np.zeros((T, self.N))

        if obs[0] == -1:
            alpha[0] = self.pi
        else:
            alpha[0] = self.pi * self.B[:, obs[0]]
        alpha[0] = alpha[0] / (alpha[0].sum() + 1e-10)

        for t in range(1, T):
            alpha[t] = alpha[t-1] @ self.A
            if obs[t] != -1:
                alpha[t] = alpha[t] * self.B[:, obs[t]]
            alpha[t] = alpha[t] / (alpha[t].sum() + 1e-10)

        return alpha

    def _backward(self, obs):
        T = len(obs)
        beta = np.zeros((T, self.N))
        beta[-1] = 1.0

        for t in range(T-2, -1, -1):
            if obs[t+1] != -1:
                beta[t] = self.A @ (self.B[:, obs[t+1]] * beta[t+1])
            else:
                beta[t] = self.A @ beta[t+1]
            beta[t] = beta[t] / (beta[t].sum() + 1e-10)

        return beta


class StableDQN(nn.Module):
    """Stable DQN with proper initialization"""

    def __init__(self):
        super().__init__()

        # Smaller network for stability
        self.fc1 = nn.Linear(52, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 26)

        # Xavier initialization
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)

        self.fc1.bias.data.zero_()
        self.fc2.bias.data.zero_()
        self.fc3.bias.data.zero_()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


class RobustAgent:
    """Debugged, stable agent"""

    def __init__(self, hmm):
        self.hmm = hmm
        self.gamma = 0.95
        self.epsilon = 1.0  # Start high
        self.eps_decay = 0.9995
        self.eps_min = 0.05

        # Networks
        self.policy = StableDQN().to(device)
        self.target = StableDQN().to(device)
        self.target.load_state_dict(self.policy.state_dict())
        self.target.eval()

        # Optimizer with smaller learning rate
        self.optimizer = optim.Adam(self.policy.parameters(), lr=0.0001)

        # Memory
        self.memory = deque(maxlen=50000)
        self.batch_size = 64

        # Stats
        self.steps = 0
        self.stats = {'wins': 0, 'games': 0, 'wrong': 0, 'rep': 0}

        # English frequency for bootstrapping
        self.freq = np.array([
            0.082, 0.015, 0.028, 0.043, 0.127, 0.022, 0.020, 0.061, 0.070,
            0.002, 0.008, 0.040, 0.024, 0.067, 0.075, 0.019, 0.001, 0.060,
            0.063, 0.091, 0.028, 0.010, 0.024, 0.002, 0.020, 0.001
        ])

    def get_state(self, pattern, guessed, wrong_left):
        """Simple 52-dim state"""
        # HMM probs (26)
        hmm = self.hmm.get_probs(pattern, guessed)

        # Guessed mask (26)
        mask = np.zeros(26)
        for l in guessed:
            mask[SYM2IDX[l]] = 1

        state = np.concatenate([hmm, mask])
        return state.astype(np.float32)

    def select_action(self, state, guessed, training=True):
        """Select action with fallback to frequency"""
        available = [i for i in range(26) if ALPHABET[i] not in guessed]
        if not available:
            return None

        # Epsilon greedy
        if training and random.random() < self.epsilon:
            # Smart exploration: use HMM + frequency
            hmm = state[:26]
            combined = 0.6 * hmm + 0.4 * self.freq
            combined = np.array([combined[i] if i in available else 0 for i in range(26)])
            if combined.sum() > 0:
                combined /= combined.sum()
                return np.random.choice(26, p=combined)
            return random.choice(available)

        # Exploit: use network
        with torch.no_grad():
            state_t = torch.FloatTensor(state).unsqueeze(0).to(device)
            q = self.policy(state_t).cpu().numpy()[0]

            # Mask unavailable
            for i in range(26):
                if i not in available:
                    q[i] = -1e9

            return np.argmax(q)

    def remember(self, s, a, r, ns, done):
        """Store transition"""
        self.memory.append((s, a, r, ns, done))

    def train_step(self):
        """Single training step"""
        if len(self.memory) < self.batch_size:
            return 0

        # Sample
        batch = random.sample(self.memory, self.batch_size)

        # Unpack and convert to numpy first (fixes warning)
        states_list, actions_list, rewards_list, next_states_list, dones_list = zip(*batch)

        states = torch.FloatTensor(np.array(states_list, dtype=np.float32)).to(device)
        actions = torch.LongTensor(actions_list).to(device)
        rewards = torch.FloatTensor(rewards_list).to(device)
        next_states = torch.FloatTensor(np.array(next_states_list, dtype=np.float32)).to(device)
        dones = torch.FloatTensor(dones_list).to(device)

        # Current Q
        q_vals = self.policy(states)
        q_vals = q_vals.gather(1, actions.unsqueeze(1)).squeeze(1)

        # Target Q
        with torch.no_grad():
            next_q = self.target(next_states).max(1)[0]
            target = rewards + (1 - dones) * self.gamma * next_q

        # Clip targets to prevent explosion
        target = torch.clamp(target, -100, 100)

        # Loss
        loss = F.mse_loss(q_vals, target)

        # Optimize
        self.optimizer.zero_grad()
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(self.policy.parameters(), 0.5)

        self.optimizer.step()

        return loss.item()

    def train_episode(self, word):
        """Play one game"""
        pattern = list('_' * len(word))
        guessed = set()
        wrong = 0
        repeated = 0

        while wrong < 6:
            # Win check
            if '_' not in pattern:
                self.stats['wins'] += 1
                return True, wrong, repeated

            # State
            state = self.get_state(''.join(pattern), guessed, 6 - wrong)

            # Action
            action = self.select_action(state, guessed, training=True)
            if action is None:
                break

            letter = ALPHABET[action]

            # Repeated?
            if letter in guessed:
                repeated += 1
                reward = -10
                self.remember(state, action, reward, state, False)
                continue

            guessed.add(letter)

            # Correct?
            if letter in word:
                count = word.count(letter)
                for i, c in enumerate(word):
                    if c == letter:
                        pattern[i] = letter
                reward = 10 * count
            else:
                wrong += 1
                reward = -5

            # Next state
            next_state = self.get_state(''.join(pattern), guessed, 6 - wrong)
            done = (wrong >= 6)

            self.remember(state, action, reward, next_state, done)

            # Train every 4 steps
            if self.steps % 4 == 0:
                self.train_step()

            self.steps += 1

            # Update target every 500 steps
            if self.steps % 500 == 0:
                self.target.load_state_dict(self.policy.state_dict())

        return False, wrong, repeated

    def train(self, words, episodes=10000):
        """Train agent"""
        print(f"\n🎓 Training for {episodes} episodes")

        # Split train/val
        random.shuffle(words)
        split = int(0.9 * len(words))
        train_words = words[:split]
        val_words = words[split:]

        best_score = -1e9

        for ep in range(episodes):
            # Sample word
            word = random.choice(train_words)

            # Train
            won, wrong, rep = self.train_episode(word)

            # Stats
            self.stats['games'] += 1
            self.stats['wrong'] += wrong
            self.stats['rep'] += rep

            # Decay epsilon
            self.epsilon = max(self.eps_min, self.epsilon * self.eps_decay)

            # Report
            if (ep + 1) % 500 == 0:
                g = self.stats['games']
                w = self.stats['wins']
                wr = w / g * 100
                avg_wrong = self.stats['wrong'] / g
                avg_rep = self.stats['rep'] / g

                print(f"\n📊 Episode {ep+1}/{episodes}")
                print(f"   Win rate: {wr:.1f}%")
                print(f"   Avg wrong: {avg_wrong:.2f}")
                print(f"   Avg rep: {avg_rep:.3f}")
                print(f"   Epsilon: {self.epsilon:.3f}")
                print(f"   Memory: {len(self.memory)}")

                # Validate
                if len(val_words) > 0:
                    score = self.validate(val_words[:100])
                    print(f"   Val score: {score:.0f}")

                    if score > best_score:
                        best_score = score
                        self.save("best_agent.pkl")
                        print(f"   ✅ Saved!")

        print(f"\n✅ Done! Final WR: {self.stats['wins']/self.stats['games']*100:.1f}%")

    def play(self, word):
        """Play without training"""
        pattern = list('_' * len(word))
        guessed = set()
        wrong = 0
        rep = 0

        while wrong < 6:
            if '_' not in pattern:
                return True, wrong, rep

            state = self.get_state(''.join(pattern), guessed, 6 - wrong)
            action = self.select_action(state, guessed, training=False)

            if action is None:
                break

            letter = ALPHABET[action]

            if letter in guessed:
                rep += 1
                continue

            guessed.add(letter)

            if letter in word:
                for i, c in enumerate(word):
                    if c == letter:
                        pattern[i] = letter
            else:
                wrong += 1

        return False, wrong, rep

    def validate(self, words):
        """Quick validation"""
        old_eps = self.epsilon
        self.epsilon = 0

        wins = 0
        total_wrong = 0
        total_rep = 0

        for word in words:
            won, wrong, rep = self.play(word)
            if won:
                wins += 1
            total_wrong += wrong
            total_rep += rep

        self.epsilon = old_eps

        sr = wins / len(words)
        score = (sr * 2000) - (total_wrong * 5) - (total_rep * 2)
        return score

    def evaluate(self, words):
        """Final eval"""
        print(f"\n📊 Evaluating {len(words)} words...")

        self.epsilon = 0
        wins = 0
        total_wrong = 0
        total_rep = 0

        for i, word in enumerate(words):
            won, wrong, rep = self.play(word)
            if won:
                wins += 1
            total_wrong += wrong
            total_rep += rep

            if (i + 1) % 500 == 0:
                print(f"   {i+1}/{len(words)}")

        sr = wins / len(words)
        score = (sr * 2000) - (total_wrong * 5) - (total_rep * 2)

        print(f"\n📈 RESULTS:")
        print(f"   Wins: {wins}/{len(words)} ({sr*100:.1f}%)")
        print(f"   Wrong: {total_wrong} (avg {total_wrong/len(words):.2f})")
        print(f"   Repeated: {total_rep} (avg {total_rep/len(words):.3f})")
        print(f"   🏆 SCORE: {score:.2f}")

        return score

    def save(self, path):
        torch.save({
            'policy': self.policy.state_dict(),
            'target': self.target.state_dict(),
            'eps': self.epsilon,
            'stats': self.stats
        }, path)

    def load(self, path):
        ckpt = torch.load(path, map_location=device)
        self.policy.load_state_dict(ckpt['policy'])
        self.target.load_state_dict(ckpt['target'])
        self.epsilon = ckpt['eps']
        self.stats = ckpt['stats']


def main():
    print("="*60)
    print("🎯 STABLE HMM + DQN AGENT")
    print("="*60)

    # Load HMM
    print("\n1️⃣ Loading HMM...")
    with open("hmm_trained.pkl", "rb") as f:
        model = pickle.load(f)

    hmm = SimpleHMM(model['pi'], model['A'], model['B'])

    # Load words
    print("\n2️⃣ Loading words...")
    with open("corpus.txt") as f:
        words = [line.strip().upper() for line in f]
    words = [''.join(c for c in w if c in SYM2IDX) for w in words if len(w) > 0]
    print(f"   {len(words)} words")

    # Train
    print("\n3️⃣ Creating agent...")
    agent = RobustAgent(hmm)

    agent.train(words, episodes=10000)

    # Test
    print("\n4️⃣ Testing...")
    with open("test.txt") as f:
        test = [line.strip().upper() for line in f]
    test = [''.join(c for c in w if c in SYM2IDX) for w in test if len(w) > 0]

    agent.evaluate(test[:2000])

    print("\n" + "="*60)


if __name__ == "__main__":
    main()

🚀 Device: cpu
🎯 STABLE HMM + DQN AGENT

1️⃣ Loading HMM...

2️⃣ Loading words...
   50000 words

3️⃣ Creating agent...

🎓 Training for 10000 episodes

📊 Episode 500/10000
   Win rate: 10.2%
   Avg wrong: 5.82
   Avg rep: 0.000
   Epsilon: 0.779
   Memory: 5265
   Val score: -2465
   ✅ Saved!

📊 Episode 1000/10000
   Win rate: 11.8%
   Avg wrong: 5.79
   Avg rep: 0.000
   Epsilon: 0.606
   Memory: 10645
   Val score: -2460
   ✅ Saved!

📊 Episode 1500/10000
   Win rate: 11.1%
   Avg wrong: 5.79
   Avg rep: 0.000
   Epsilon: 0.472
   Memory: 16038
   Val score: -2415
   ✅ Saved!

📊 Episode 2000/10000
   Win rate: 11.4%
   Avg wrong: 5.78
   Avg rep: 0.000
   Epsilon: 0.368
   Memory: 21547
   Val score: -2745

📊 Episode 2500/10000
   Win rate: 11.3%
   Avg wrong: 5.78
   Avg rep: 0.000
   Epsilon: 0.286
   Memory: 27153
   Val score: -2755

📊 Episode 3000/10000
   Win rate: 10.9%
   Avg wrong: 5.79
   Avg rep: 0.000
   Epsilon: 0.223
   Memory: 32623
   Val score: -2900

📊 Episode 3500/10

In [ ]:
import numpy as np
import string
import pickle
from collections import Counter, defaultdict
import random

ALPHABET = list(string.ascii_uppercase)
SYM2IDX = {c: i for i, c in enumerate(ALPHABET)}
IDX2SYM = {i: c for c, i in SYM2IDX.items()}


class HMMFeatureExtractor:
    """
    Extracts HMM-based features for RL state representation
    """

    def __init__(self, pi, A, B):
        self.pi = pi
        self.A = A
        self.B = B
        self.N = len(pi)

    def get_letter_probabilities(self, pattern, guessed_letters):
        """
        Get HMM probability distribution over all 26 letters
        Returns: vector of 26 probabilities
        """
        length = len(pattern)

        # Convert pattern to observation indices (-1 for unknown)
        obs = []
        unknown_positions = []
        for i, c in enumerate(pattern):
            if c == '_':
                obs.append(-1)
                unknown_positions.append(i)
            else:
                obs.append(SYM2IDX[c])

        if not unknown_positions:
            return np.zeros(26)

        # Run forward-backward
        alpha = self._forward(obs)
        beta = self._backward(obs)

        # Compute marginal probabilities for each letter
        letter_probs = np.zeros(26)

        for pos in unknown_positions:
            gamma = alpha[pos] * beta[pos]
            gamma = gamma / (np.sum(gamma) + 1e-300)

            # Accumulate emission probabilities
            for letter_idx in range(26):
                letter_probs[letter_idx] += np.sum(gamma * self.B[:, letter_idx])

        # Normalize
        total = np.sum(letter_probs)
        if total > 0:
            letter_probs = letter_probs / total

        # Zero out already guessed letters
        for letter in guessed_letters:
            letter_probs[SYM2IDX[letter]] = 0

        return letter_probs

    def get_state_distribution(self, pattern):
        """
        Get HMM hidden state distribution given pattern
        Returns: vector of state probabilities
        """
        obs = []
        for c in pattern:
            if c == '_':
                obs.append(-1)
            else:
                obs.append(SYM2IDX[c])

        alpha = self._forward(obs)
        return alpha[-1] / (np.sum(alpha[-1]) + 1e-300)

    def _forward(self, obs):
        """Forward algorithm with unknown observations"""
        T = len(obs)
        alpha = np.zeros((T, self.N))

        # Initialize
        if obs[0] == -1:
            alpha[0] = self.pi
        else:
            alpha[0] = self.pi * self.B[:, obs[0]]
        alpha[0] = alpha[0] / (np.sum(alpha[0]) + 1e-300)

        # Forward pass
        for t in range(1, T):
            alpha[t] = alpha[t - 1] @ self.A
            if obs[t] != -1:
                alpha[t] = alpha[t] * self.B[:, obs[t]]
            alpha[t] = alpha[t] / (np.sum(alpha[t]) + 1e-300)

        return alpha

    def _backward(self, obs):
        """Backward algorithm with unknown observations"""
        T = len(obs)
        beta = np.zeros((T, self.N))
        beta[-1] = 1.0

        for t in range(T - 2, -1, -1):
            if obs[t + 1] != -1:
                beta[t] = self.A @ (self.B[:, obs[t + 1]] * beta[t + 1])
            else:
                beta[t] = self.A @ beta[t + 1]
            beta[t] = beta[t] / (np.sum(beta[t]) + 1e-300)

        return beta


class HangmanQLearningAgent:
    """
    Q-Learning agent for Hangman that uses HMM features
    """

    def __init__(self, hmm_extractor, learning_rate=0.1, discount=0.95,
                 epsilon=0.1, epsilon_decay=0.995, epsilon_min=0.01):
        self.hmm = hmm_extractor
        self.alpha = learning_rate
        self.gamma = discount
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min

        # Q-table: state -> action -> value
        self.Q = defaultdict(lambda: np.zeros(26))

        # For tracking performance
        self.training_stats = {
            'games_played': 0,
            'games_won': 0,
            'total_wrong_guesses': 0,
            'total_repeated_guesses': 0
        }

    def get_state_key_simple(self, pattern, guessed_letters, wrong_guesses_left):
        """
        Simpler state representation for smaller state space
        """
        # Get HMM probabilities
        hmm_probs = self.hmm.get_letter_probabilities(pattern, guessed_letters)

        # Discretize HMM probs into bins
        top_3_indices = np.argsort(hmm_probs)[-3:][::-1]
        top_3_letters = tuple(ALPHABET[i] for i in top_3_indices)

        # Pattern features
        length = len(pattern)
        known_count = sum(1 for c in pattern if c != '_')
        known_ratio = int(known_count / length * 10)  # 0-10

        state_key = (
            length,
            known_ratio,
            wrong_guesses_left,
            top_3_letters,
            tuple(sorted(guessed_letters))
        )

        return state_key

    def choose_action(self, state_key, guessed_letters, hmm_probs, training=True):
        """
        Choose action using epsilon-greedy with HMM guidance
        """
        available_letters = [i for i in range(26) if ALPHABET[i] not in guessed_letters]

        if not available_letters:
            return None

        # Epsilon-greedy exploration
        if training and random.random() < self.epsilon:
            # Explore: weighted random by HMM probs
            probs = np.array([hmm_probs[i] for i in available_letters])
            if np.sum(probs) > 0:
                probs = probs / np.sum(probs)
                action = np.random.choice(available_letters, p=probs)
            else:
                action = random.choice(available_letters)
        else:
            # Exploit: choose best Q-value among available
            q_values = self.Q[state_key]

            # Combine Q-values with HMM probs for better decisions
            combined_scores = np.zeros(26)
            for i in available_letters:
                combined_scores[i] = 0.7 * q_values[i] + 0.3 * hmm_probs[i]

            action = np.argmax(combined_scores)

            # If Q-value is 0 (unvisited), fall back to HMM
            if q_values[action] == 0:
                action = np.argmax(hmm_probs)

        return action

    def update_q_value(self, state, action, reward, next_state, done):
        """
        Q-learning update: Q(s,a) += α[r + γ max Q(s',a') - Q(s,a)]
        """
        current_q = self.Q[state][action]

        if done:
            max_next_q = 0
        else:
            max_next_q = np.max(self.Q[next_state])

        new_q = current_q + self.alpha * (reward + self.gamma * max_next_q - current_q)
        self.Q[state][action] = new_q

    def train_episode(self, word, max_wrong=6):
        """
        Train on a single game of Hangman with reward shaping
        """
        pattern = ['_'] * len(word)
        guessed_letters = set()
        wrong_guesses = 0
        repeated_guesses = 0

        episode_history = []  # (state, action, reward)

        while wrong_guesses < max_wrong:
            if '_' not in pattern:
                break

            # Get current state
            pattern_str = ''.join(pattern)
            hmm_probs = self.hmm.get_letter_probabilities(pattern_str, guessed_letters)
            state = self.get_state_key_simple(pattern_str, guessed_letters, max_wrong - wrong_guesses)

            # Choose action
            action = self.choose_action(state, guessed_letters, hmm_probs, training=True)
            if action is None:
                break

            letter = ALPHABET[action]
            reward = 0

            # --- Reward shaping logic ---
            if letter in guessed_letters:
                repeated_guesses += 1
                reward = -8
                episode_history.append((state, action, reward))
                continue

            guessed_letters.add(letter)

            if letter in word:
                new_letters = sum(1 for i, c in enumerate(word) if c == letter and pattern[i] == '_')
                for i, c in enumerate(word):
                    if c == letter:
                        pattern[i] = letter
                reward = 10 * new_letters
            else:
                wrong_guesses += 1
                reward = -4

            if '_' not in pattern:
                reward += 50
            elif wrong_guesses >= max_wrong:
                reward -= 50

            episode_history.append((state, action, reward))

            if wrong_guesses >= max_wrong or '_' not in pattern:
                break

        self._backpropagate_rewards(episode_history, reward)

        won = '_' not in pattern
        if won:
            self.training_stats['games_won'] += 1

        return won, wrong_guesses, repeated_guesses

    def _backpropagate_rewards(self, episode_history, final_reward):
        """
        Backpropagate rewards through episode
        """
        for i, (state, action, immediate_reward) in enumerate(episode_history):
            remaining_steps = len(episode_history) - i - 1
            discounted_final = final_reward * (self.gamma ** remaining_steps)
            total_reward = immediate_reward + discounted_final

            if i < len(episode_history) - 1:
                next_state = episode_history[i + 1][0]
                done = False
            else:
                next_state = state
                done = True

            self.update_q_value(state, action, total_reward, next_state, done)

    def train(self, word_list, episodes=5000):
        """
        Train the Q-learning agent on a list of words
        """
        print(f"\n🎓 Training Q-Learning Agent for {episodes} episodes...")
        print(f"   Word list size: {len(word_list)}")

        for episode in range(episodes):
            word = random.choice(word_list).upper()
            word = ''.join([c for c in word if c in SYM2IDX])

            if len(word) == 0:
                continue

            won, wrong, repeated = self.train_episode(word)

            self.training_stats['games_played'] += 1
            self.training_stats['total_wrong_guesses'] += wrong
            self.training_stats['total_repeated_guesses'] += repeated

            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

            if (episode + 1) % 500 == 0:
                games = self.training_stats['games_played']
                wins = self.training_stats['games_won']
                win_rate = wins / games * 100 if games > 0 else 0
                avg_wrong = self.training_stats['total_wrong_guesses'] / games
                avg_repeated = self.training_stats['total_repeated_guesses'] / games

                print(f"\n   Episode {episode + 1}/{episodes}")
                print(f"   Win rate: {win_rate:.2f}%")
                print(f"   Avg wrong guesses: {avg_wrong:.2f}")
                print(f"   Avg repeated guesses: {avg_repeated:.2f}")
                print(f"   Epsilon: {self.epsilon:.4f}")
                print(f"   Q-table size: {len(self.Q)}")

        print(f"\n✅ Training complete!")
        print(f"   Final win rate: {self.training_stats['games_won'] / self.training_stats['games_played'] * 100:.2f}%")
        print(f"   Q-table states: {len(self.Q)}")

    def play_game(self, word, max_wrong=6, verbose=False):
        """
        Play a single game (testing/evaluation)
        """
        pattern = ['_'] * len(word)
        guessed_letters = set()
        wrong_guesses = 0
        repeated_guesses = 0

        if verbose:
            print(f"\n🎮 Playing: {word}")

        while wrong_guesses < max_wrong:
            if '_' not in pattern:
                if verbose:
                    print(f"   ✅ WON! Wrong: {wrong_guesses}, Repeated: {repeated_guesses}")
                return True, wrong_guesses, repeated_guesses

            pattern_str = ''.join(pattern)
            hmm_probs = self.hmm.get_letter_probabilities(pattern_str, guessed_letters)
            state = self.get_state_key_simple(pattern_str, guessed_letters, max_wrong - wrong_guesses)

            action = self.choose_action(state, guessed_letters, hmm_probs, training=False)
            if action is None:
                break

            letter = ALPHABET[action]

            if letter in guessed_letters:
                repeated_guesses += 1
                if verbose:
                    print(f"   ❌ Repeated: {letter}")
                continue

            guessed_letters.add(letter)

            if letter in word:
                for i, c in enumerate(word):
                    if c == letter:
                        pattern[i] = letter
                if verbose:
                    print(f"   ✓ {letter}: {''.join(pattern)}")
            else:
                wrong_guesses += 1
                if verbose:
                    print(f"   ✗ {letter}: {''.join(pattern)} ({wrong_guesses}/{max_wrong})")

        if verbose:
            print(f"   ❌ LOST! Wrong: {wrong_guesses}, Repeated: {repeated_guesses}")
        return False, wrong_guesses, repeated_guesses

    def evaluate(self, test_words, max_wrong=6):
        """
        Evaluate agent on test set
        """
        print(f"\n📊 Evaluating on {len(test_words)} words...")

        wins = 0
        total_wrong = 0
        total_repeated = 0

        for word in test_words:
            won, wrong, repeated = self.play_game(word, max_wrong)
            if won:
                wins += 1
            total_wrong += wrong
            total_repeated += repeated

        success_rate = wins / len(test_words)
        final_score = (success_rate * 2000) - (total_wrong * 5) - (total_repeated * 2)

        print(f"\n📈 RESULTS:")
        print(f"   Games played: {len(test_words)}")
        print(f"   Wins: {wins}")
        print(f"   Success rate: {success_rate * 100:.2f}%")
        print(f"   Total wrong guesses: {total_wrong}")
        print(f"   Total repeated guesses: {total_repeated}")
        print(f"   Avg wrong per game: {total_wrong / len(test_words):.2f}")
        print(f"   Avg repeated per game: {total_repeated / len(test_words):.2f}")
        print(f"\n   🏆 FINAL SCORE: {final_score:.2f}")

        return success_rate, total_wrong, total_repeated, final_score

    def save(self, path="q_learning_agent.pkl"):
        """Save Q-table and parameters"""
        data = {
            'Q': dict(self.Q),
            'alpha': self.alpha,
            'gamma': self.gamma,
            'epsilon': self.epsilon,
            'stats': self.training_stats
        }
        with open(path, 'wb') as f:
            pickle.dump(data, f)
        print(f"✅ Saved agent to {path}")

    def load(self, path="q_learning_agent.pkl"):
        """Load Q-table and parameters"""
        with open(path, 'rb') as f:
            data = pickle.load(f)
        self.Q = defaultdict(lambda: np.zeros(26), data['Q'])
        self.alpha = data['alpha']
        self.gamma = data['gamma']
        self.epsilon = data['epsilon']
        self.training_stats = data['stats']
        print(f"✅ Loaded agent from {path}")


# ========== MAIN USAGE ==========
def main():
    print("=" * 60)
    print("🤖 HMM + Q-Learning Hangman Agent")
    print("=" * 60)

    # Load HMM model
    print("\n1️⃣ Loading HMM model...")
    with open("hmm_trained.pkl", "rb") as f:
        model = pickle.load(f)
    pi, A, B = model['pi'], model['A'], model['B']

    # Create HMM feature extractor
    hmm_extractor = HMMFeatureExtractor(pi, A, B)

    # Load training words
    print("\n2️⃣ Loading training words...")
    with open("corpus.txt", 'r', encoding='utf-8') as f:
        train_words = [line.strip().upper() for line in f]
    train_words = [''.join([c for c in w if c in SYM2IDX]) for w in train_words]
    train_words = [w for w in train_words if len(w) > 0]
    print(f"   Loaded {len(train_words)} training words")

    # Create Q-learning agent
    agent = HangmanQLearningAgent(
        hmm_extractor,
        learning_rate=0.1,
        discount=0.95,
        epsilon=0.3,
        epsilon_decay=0.995,
        epsilon_min=0.01
    )

    # Train agent
    print("\n3️⃣ Training agent...")
    agent.train(train_words, episodes=10000)

    # Save agent
    agent.save("hangman_qlearning_agent_v2.pkl")

    # Load test words
    print("\n4️⃣ Loading test words...")
    with open("test.txt", 'r', encoding='utf-8') as f:
        test_words = [line.strip().upper() for line in f]
    test_words = [''.join([c for c in w if c in SYM2IDX]) for w in test_words]
    test_words = [w for w in test_words if len(w) > 0]
    print(f"   Loaded {len(test_words)} test words")

    # Evaluate
    print("\n5️⃣ Evaluating agent...")
    agent.evaluate(test_words[:2000], max_wrong=6)

    print("\n" + "=" * 60)


if __name__ == "__main__":
    main()


🤖 HMM + Q-Learning Hangman Agent

1️⃣ Loading HMM model...

2️⃣ Loading training words...
   Loaded 50000 training words

3️⃣ Training agent...

🎓 Training Q-Learning Agent for 10000 episodes...
   Word list size: 50000

   Episode 500/10000
   Win rate: 33.60%
   Avg wrong guesses: 5.23
   Avg repeated guesses: 0.00
   Epsilon: 0.0245
   Q-table size: 4495

   Episode 1000/10000
   Win rate: 31.70%
   Avg wrong guesses: 5.25
   Avg repeated guesses: 0.00
   Epsilon: 0.0100
   Q-table size: 8497

   Episode 1500/10000
   Win rate: 32.13%
   Avg wrong guesses: 5.24
   Avg repeated guesses: 0.00
   Epsilon: 0.0100
   Q-table size: 12384

   Episode 2000/10000
   Win rate: 31.80%
   Avg wrong guesses: 5.25
   Avg repeated guesses: 0.00
   Epsilon: 0.0100
   Q-table size: 16009

   Episode 2500/10000
   Win rate: 31.96%
   Avg wrong guesses: 5.24
   Avg repeated guesses: 0.00
   Epsilon: 0.0100
   Q-table size: 19404

   Episode 3000/10000
   Win rate: 32.63%
   Avg wrong guesses: 5.23
   